<a href="https://colab.research.google.com/github/Karenw1004/TreasureHunt/blob/main/notebooks/TreasureHunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import output
from tqdm import tqdm

# Initialize the map without starting position 

In [ ]:
#        0    1   2   3   4   5   6   7   8   9   10
maze = [["W","W","W","W","W","E","W","W","W","W","W"],
        ["W","R","R","R","R","R","R","R","R","R","W"],
        ["W","R","W","W","W","W","W","R","W","R","W"],
        ["W","R","R","R","R","R","R","R","W","R","W"],
        ["W","W","W","R","W","W","W","R","W","W","W"],
        ["R","R","R","R","R","R","R","R","R","R","R"],
        ["W","W","W","W","W","R","W","W","W","W","W"],
        ["W","R","R","R","R","R","R","R","R","R","W"],
        ["W","W","W","R","W","W","W","R","W","W","W"],
        ["R","R","R","R","R","R","R","R","R","R","R"],
        ["W","W","W","W","W","W","W","W","W","W","W"]]

In [ ]:
maze_row = len(maze)
maze_col = len(maze[0])

Make a mapping dictionary with chars as key and tuple key

> "C": (emoji-unicode, rewards)

emoji-unicode is from <br/>
https://unicode.org/emoji/charts/emoji-list.html

In [ ]:
mapping = {
    # Road
    "R": ("\N{white large square}", -1),
    # END
    "E": ("\N{party popper}", 1000) ,
    # Wall
    "W": ("\N{brick}", -100),
    # Human
    "H": ("\N{girl}", None),
    # Bomb
    "B": ("\N{bomb}", -100) ,
    # Gold
    "G": ("\N{money bag}", 30)
}

In [ ]:
def print_maze(human_pos):
  for i in range(len(maze)):
    for j in range(len(maze[0])):
      if ((i == human_pos[0]) and (j == human_pos[1])):
        val = "H"
      else:
        val = maze[i][j]
      emoji = mapping[val][0]
      print(emoji,end=" " )
    print("\n")


In [ ]:
print_maze((maze_row+1,maze_col+1))

🧱 🧱 🧱 🧱 🧱 🎉 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 ⬜ 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 



maze looks good

# Q table with n cols, m rows, number_of_actions
0. Up
1. Right
2. Down
3. Left

In [ ]:
q_table = np.zeros((maze_row, maze_col,  4))

# Next action should be the max reward

In [ ]:
def get_next_action(row, col, epilson):
  if np.random.random() < epilson:
    return np.argmax(q_table[row,col])
  else:
    return np.random.randint(4)

# Movement of the suggested action

In [ ]:
def get_next_step(row, col,step_index):
  up_right_down_left = [(row-1,col),(row,col+1),(row+1,col),(row,col-1)]
  x, y = up_right_down_left[step_index]
  if 0 <= x < maze_row and 0 <= y < maze_col:
    return up_right_down_left[step_index]
  else:
    return (row, col) # No change

## Terminal state is the end of an episode.
- When you crash the wall, GAME OVER
- When you reach the end, DONE


In [ ]:
terminal_state = ["W","E"]
def is_terminal_state(row, col):
  return maze[row][col] in terminal_state

In [ ]:
is_terminal_state(0,0)

True

In [ ]:
def get_random_starting_pos():
  while True:
    row = np.random.randint(maze_row)
    col = np.random.randint(maze_col)
    if not is_terminal_state(row,col):
      return (row,col)

In [ ]:
get_random_starting_pos()

(7, 1)

# Training

In [ ]:
epilson = 0.9
discount_factor = 0.9
learning_rate = 0.9
epochs = 1000

$
\underbrace{\text{New}Q(s,a)}_{\scriptstyle\text{New Q-Value}} =
\underbrace{Q(s,a)}_{\scriptstyle\text{Old Q-Value}}+
\mkern-34mu\underset{\text{Learning Rate}}{\underset{\Bigl|}{\alpha}}\mkern-30mu[\underbrace{R(s,a)}_{\scriptstyle\text{Reward}}+
\mkern-30mu\underset{\text{Discount rate}}{\underset{\Biggl|}{\gamma}}\mkern-75mu\overbrace{\max Q'(s',a')}^{\scriptstyle\substack{\text{Maximum predicted reward, given} \\ \text{new state and all possible actions}}}\mkern-45mu-Q(s,a)]
$

In [ ]:
def train():
  for _ in tqdm(range(epochs)):
    row, col = get_random_starting_pos()

    while not is_terminal_state(row,col):
      action_index = get_next_action(row,col, epilson)

      old_row, old_col = row , col
      row , col = get_next_step(row,col,action_index)

      reward = mapping[ maze[row][col] ][1]
      old_q_val = q_table[old_row, old_col, action_index]
      temporal_diff = reward + (discount_factor * np.max(q_table[row, col])) - old_q_val
      
      new_q_val = old_q_val + (learning_rate * temporal_diff)
      q_table[old_row, old_col, action_index] = new_q_val

  print("\nTraining done!")

In [ ]:
train()

100%|██████████| 1000/1000 [00:00<00:00, 5661.48it/s]


Training done!


In [ ]:
def get_shortest_path(row, col):
  if is_terminal_state(row, col):
    return []
  else: 
    cur_row , cur_col = row, col
    shortest_path = []
    shortest_path.append((cur_row, cur_col))
    while not is_terminal_state(cur_row, cur_col):
      action_index = get_next_action(cur_row, cur_col, 1.)
      cur_row, cur_col = get_next_step(cur_row, cur_col, action_index)
      shortest_path.append((cur_row, cur_col))
    return shortest_path

In [ ]:
def print_movement(path):
  for i in range(len(path)):
    print(f"Step {i}")
    coords = path[i]
    print_maze(coords)

In [ ]:
human_pos = (9,0)
path = get_shortest_path(human_pos[0], human_pos[1])
print_movement(path)


Step 0
🧱 🧱 🧱 🧱 🧱 🎉 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 ⬜ 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

👧 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 

Step 1
🧱 🧱 🧱 🧱 🧱 🎉 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 ⬜ 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ 👧 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 

Step 2
🧱 🧱 🧱 🧱 🧱 🎉 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 ⬜ 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ 👧 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 

Step 3
🧱 🧱 🧱 🧱 🧱 🎉 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 

🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 ⬜ 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🧱 ⬜ 🧱 

🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 

⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 

🧱 🧱 🧱 🧱 🧱 ⬜ 🧱 🧱 🧱 🧱 🧱 

🧱 ⬜ ⬜ ⬜ ⬜ ⬜ 

You can see that we successfully train to reach the goal without crashing the wall

# Future todo
- Add gold to gain points
- Add bomb to lose health